In [ ]:
!pip install mtcnn

In [ ]:
try:
    from google.colab import drive

    drive.mount("/content/drive")
    import zipfile

    with zipfile.ZipFile("/content/drive/MyDrive/ML/train.zip", "r") as zip_ref:
        zip_ref.extractall("./")
    with zipfile.ZipFile("/content/drive/MyDrive/ML/test.zip", "r") as zip_ref:
        zip_ref.extractall("./")
    trainDataFolderPath = "/content/train/"
    testDataFolderPath = "/content/test/"
except:
    trainDataFolderPath = "Data/train/"
    testDataFolderPath = "Data/test/"
    print("Using Local Machine")

# try:
#     from google.colab import drive

#     drive.mount("/content/drive")
#     import zipfile

#     with zipfile.ZipFile(
#         "/content/drive/MyDrive/ML/Transformed Train.zip", "r"
#     ) as zip_ref:
#         zip_ref.extractall("./")
#     with zipfile.ZipFile(
#         "/content/drive/MyDrive/ML/Transformed Test.zip", "r"
#     ) as zip_ref:
#         zip_ref.extractall("./")
#     trainDataFolderPath = "/content/Transformed Train/"
#     testDataFolderPath = "/content/Transformed Test/"
# except:
#     trainDataFolderPath = "Data/Transformed Train/"
    # testDataFolderPath = "Data/Transformed Test/"
    # print("Using Local Machine")

In [ ]:
import os
import gc
import cv2
import copy
import json
import numpy as np
from tqdm import tqdm
from mtcnn.mtcnn import MTCNN
from itertools import product
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, accuracy_score

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import datetime
import tensorflow as tf

from keras.models import Sequential, load_model
from keras.utils import to_categorical
from keras.optimizers import RMSprop

from keras.layers import (
    Conv2D,
    BatchNormalization,
    MaxPooling2D,
    Dropout,
    Flatten,
    Activation,
    Dense,
)
from keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
physicalDevices = tf.config.list_physical_devices("GPU")
print(physicalDevices)
if len(physicalDevices) > 0:
    tf.config.experimental.set_memory_growth(physicalDevices[0], True)

In [ ]:
%reload_ext tensorboard
modelPath = 'SavedModels/model1.sav'
logsDir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboardCBK = TensorBoard(log_dir=logsDir, histogram_freq=1)
earlyStoppingCBK = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=200)
modelCBK = ModelCheckpoint(
    modelPath+'.mcp.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduceLRPlateauCBK  = ReduceLROnPlateau(monitor='val_loss',
                                            patience=200,
                                            verbose=1,
                                            factor=0.2)

callbacks = [earlyStoppingCBK,
             reduceLRPlateauCBK, tensorboardCBK]


In [ ]:
def ShowImage(imageName: str, image: np.ndarray):
    plt.imshow(image)
    plt.title(imageName)
    plt.axis("off")
    plt.show()

In [ ]:
def DrawBoundaryBoxs(
    frame: np.ndarray,
    boundryBox: list,
    color: tuple = (0, 255, 0),
    thickness: int = 2,
):
    [x, y, w, h] = boundryBox
    frame = cv2.rectangle(frame, (x, y), (x + w, y + h), color, thickness)
    return frame

In [ ]:
def DisplayPrediction(frame: np.ndarray, Name: str = None):
    if(not Name):
        Name =  "No Face Detected"
    frame_width = frame.shape[:2]
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (0, 255, 0)  # Green color
    line_type = 2
    text_size = cv2.getTextSize(Name, font, font_scale, line_type)[0]
    text_x = (frame_width - text_size[0]) // 2  # Centered horizontally
    text_y = 30
    frame = cv2.putText(frame, Name, (text_x, text_y), font, font_scale, font_color, line_type)
    return frame

In [ ]:
def DetectFaces(frame: np.ndarray, faceCascade: MTCNN):
    # grayScaleImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detectedFacesBBoxs = faceCascade.detect_faces(frame)

    detectedFacesBBoxs = [detectedFacesBBox['box'] for detectedFacesBBox in detectedFacesBBoxs]
    return detectedFacesBBoxs

In [ ]:
def CropImage(image: np.ndarray, bBox: list):
    [x, y, w, h] = bBox
    croppedImage = image[y : y + h, x : x + w]
    return croppedImage

In [ ]:
def ResizeImage(image: np.ndarray, resize: tuple = (100, 100)):
    image = cv2.resize(image, resize)
    return image

In [ ]:
def LoadDataSet(folderPath: str):
    if not (os.path.exists(folderPath)):
        print("Please ensure that FolderPath is valid")
        return None
    labelNames = os.listdir(folderPath)
    imageDataset = []
    labelDataset = []
    for folderName in labelNames:
        if not folderName == ".DS_Store":
            imageList = os.listdir(folderPath + folderName)
            for imageName in tqdm(imageList):
                if not imageName == ".DS_Store":
                    imageDataset.append(
                        # ResizeImage(
                            cv2.imread(folderPath + folderName + "/" + imageName)
                        # )
                    )
                    labelDataset.append(folderName)

    return imageDataset, labelDataset

In [ ]:
def GetMaxAreabBox(bBoxes: list, imageArea: int):
    bestbBox = []
    maxArea = 0
    for bBox in bBoxes:
        [x, y, w, h] = bBox
        if w * h > maxArea:
            maxArea = w * h
            bestbBox = bBox
    return bestbBox

In [ ]:
def TransformFaces(imageDataset: list, labelDataset: list, min_face_size: int = 20):
    faceCascade = MTCNN(min_face_size=min_face_size)
    faceImages = []
    labels = []
    zipped = zip(imageDataset, labelDataset)
    for image, label in tqdm(zipped):
        bBoxes = DetectFaces(image, faceCascade)
        imageArea = image.shape[0] * image.shape[1]

        if len(bBoxes):
            bBox = GetMaxAreabBox(bBoxes, imageArea)
            if len(bBox):
                faceImage = CropImage(image, bBox)
                faceImages.append(faceImage)
                labels.append(label)
    return faceImages, labels

In [ ]:
# trainingImageDataset, trainingLabelDataset = LoadDataSet(trainDataFolderPath)
# combined = list(zip(trainingImageDataset, trainingLabelDataset))
# np.random.shuffle(combined)
# trainingImageDataset, trainingLabelDataset = zip(*combined)

In [ ]:
trainingImageDataset, trainingLabelDataset = LoadDataSet(trainDataFolderPath)
ShowImage(trainingLabelDataset[0],trainingImageDataset[0])

In [ ]:
trainingImageDataset, trainingLabelDataset = TransformFaces(
    trainingImageDataset, trainingLabelDataset
)
# ShowImage(trainingLabelDataset[0],trainingImageDataset[0])


In [ ]:

ShowImage(trainingLabelDataset[0],trainingImageDataset[0])

In [ ]:
trainingImageDataset = [ResizeImage(image) for image in tqdm(trainingImageDataset)]
encoder = LabelEncoder()
encoder = encoder.fit(trainingLabelDataset)
trainingLabelDataset = encoder.transform(trainingLabelDataset)
trainingLabelDataset = to_categorical(trainingLabelDataset, num_classes=4)
trainingImageDataset = np.array(trainingImageDataset)
trainingLabelDataset = np.array(trainingLabelDataset)
numClasses = len(encoder.classes_)

In [ ]:
print(numClasses)
print(trainingImageDataset.shape)
print(trainingLabelDataset.shape)

In [ ]:
testImageDataset, testLabelDataset = LoadDataSet(testDataFolderPath)
testImageDataset, testLabelDataset = TransformFaces(testImageDataset, testLabelDataset)
testImageDataset = [ResizeImage(image) for image in tqdm(testImageDataset)]
testLabelDataset = encoder.transform(testLabelDataset)
testLabelDataset = to_categorical(testLabelDataset, num_classes=numClasses)
testImageDataset = np.array(testImageDataset)
testLabelDataset = np.array(testLabelDataset)

In [ ]:
print(testImageDataset.shape)
print(testLabelDataset.shape)

In [ ]:
def CreateModel(inputShape: tuple = (100, 100, 3), numClasses: int = 4):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=inputShape))
    model.add(BatchNormalization())  # ----------------
    model.add(Conv2D(64, kernel_size=3, activation="relu"))
    model.add(BatchNormalization())  # ----------------
    model.add(Conv2D(64, kernel_size=5, padding="same", activation="relu"))
    model.add(BatchNormalization())  # ----------------
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))  # ----------------

    model.add(Conv2D(128, kernel_size=3, activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=3, activation="relu"))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=5, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=3, activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Dense(128))
    model.add(BatchNormalization())

    model.add(Dense(numClasses, activation="softmax"))
    learning_rate = 0.001
    optimizer = RMSprop(lr=learning_rate)
    # optimizer="adam"

    model.compile(
        loss="categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"],
    )
    return model

In [ ]:
epochs = 100
batch_size = 256
validation_steps = 50
steps_per_epoch = trainingImageDataset.shape[0] // batch_size

In [ ]:
model = CreateModel(numClasses= numClasses)
model.summary()

In [ ]:
history = model.fit(
    trainingImageDataset,
    trainingLabelDataset,
    batch_size=32,
    epochs=epochs,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=1,
)

In [ ]:
# datagen = ImageDataGenerator(
#     rescale=1.0 / 255.0,
#     rotation_range=10,
#     width_shift_range=0.25,
#     height_shift_range=0.25,
#     shear_range=0.1,
#     zoom_range=0.25,
#     horizontal_flip=False,
# )
# trainingDataset = datagen.flow(trainingImageDataset, trainingLabelDataset)
# testDataset = datagen.flow(testImageDataset, testLabelDataset)

In [ ]:
# history = model.fit_generator(
#     trainingDataset,
#     steps_per_epoch=steps_per_epoch,
#     epochs=epochs,
#     validation_data=testDataset,
#     validation_steps=validation_steps,
#     callbacks=callbacks,
#     verbose=1,
# )

In [ ]:
print(testImageDataset.shape)

In [ ]:
testImageDataset, testLabelDataset = LoadDataSet(testDataFolderPath)
testImageDataset, testLabelDataset = TransformFaces(testImageDataset, testLabelDataset)
testImageDataset = [ResizeImage(image) for image in tqdm(testImageDataset)]
testLabelDataset = encoder.transform(testLabelDataset)
testLabelDataset = to_categorical(testLabelDataset, num_classes=numClasses)
testImageDataset = np.array(testImageDataset)
testLabelDataset = np.array(testLabelDataset)

In [ ]:
accuracy = model.evaluate(testImageDataset, testLabelDataset)
print("accuracy:", accuracy[1])


In [ ]:
for images in testImageDataset:
    prediction = model.predict(images.reshape(1, 100, 100, 3))
    print(np.argmax(prediction))
    prediction = encoder.inverse_transform([np.argmax(prediction)])
    ShowImage(prediction[0], images)

In [ ]:
model.save('MyModel.h5')